In [1]:
import pandas as pd

In [2]:
df = pd.read_csv("../data/fastball_spin_rates_2021.csv")
df.shape

(4845, 4)

In [12]:
df.columns

Index(['pitcher_name', 'game_date', 'avg_spin_rate', 'n_pitches'], dtype='object')

In [3]:
df.head()

,pitcher_name,game_date,avg_spin_rate,n_pitches
0,"Wainwright, Adam",2021-04-03,2127.415000,12
1,"Wainwright, Adam",2021-04-08,2179.723000,11
2,"Wainwright, Adam",2021-04-14,2297.968571,7
3,"Wainwright, Adam",2021-04-20,2159.150000,13
4,"Wainwright, Adam",2021-04-26,2314.515455,11


In [16]:
df.groupby("pitcher_name").count()

,game_date,avg_spin_rate,n_pitches
pitcher_name,,,
"Adon, Joan",1,1,1
"Albers, Andrew",5,5,5
"Alcantara, Sandy",32,32,32
"Alexander, Tyler",37,37,37
"Alexy, AJ",5,5,5
...,...,...,...
"Yarbrough, Ryan",6,6,6
"Ynoa, Huascar",18,18,18
"Zerpa, Angel",1,1,1


In [4]:
df.isna().sum()

pitcher_name     0
game_date        0
avg_spin_rate    7
n_pitches        0
dtype: int64

In [8]:
df['pitcher_name'].nunique(), df['game_date'].nunique()
(262, 182)

(262, 182)

In [9]:
df["avg_spin_rate"].describe()

count    4838.000000
mean     2262.381625
std       169.895006
min      1642.870000
25%      2146.792611
50%      2263.186250
75%      2378.541008
max      2899.950000
Name: avg_spin_rate, dtype: float64

In [10]:
df["n_pitches"].describe()

count    4845.000000
mean       26.978328
std        16.687643
min         1.000000
25%        12.000000
50%        27.000000
75%        40.000000
max        82.000000
Name: n_pitches, dtype: float64